In [2]:
import sqlite3
import numpy as np
import pandas as pd

In [3]:
conn=sqlite3.connect('sample_rental.db')
cursor=conn.cursor()

* buscar actores con mismo nombre y apellido y diferente actor_id

In [4]:
q=(""" 
 SELECT actor_id, first_name, last_name
 FROM actor
 WHERE (first_name, last_name) 
    in(
    SELECT first_name, last_name
    FROM actor
    GROUP BY first_name, last_name
    HAVING COUNT (actor_id)>1
    )
"""
)
df=pd.read_sql(q,conn)
df

,actor_id,first_name,last_name
0,101,SUSAN,DAVIS
1,110,SUSAN,DAVIS


* contar actores por cada pelicula

In [5]:
q=""" 
    WITH t1 AS(
    SELECT title, first_name, last_name ||""|| last_name as full_name
    FROM film
    INNER JOIN film_actor
    ON film.film_id = film_actor.film_id
    INNER JOIN actor
    ON actor.actor_id = film_actor.actor_id
    GROUP BY title, first_name, last_name
    ORDER By title
    )
    SELECT 
        title, 
        count(full_name) as count_of_actors
    FROM t1
    GROUP BY 1  
    ORDER BY 2 DESC
    LIMIT 10
"""

df=pd.read_sql(q,conn)
df

,title,count_of_actors
0,LAMBS CINCINATTI,15
1,RANDOM GO,13
2,MUMMY CREATURES,13
3,DRACULA CRYSTAL,13
4,CRAZY HOME,13
5,CHITTY LOCK,13
6,BOONDOCK BALLROOM,13
7,TITANIC BOONDOCK,12
8,SKY MIRACLE,12
9,LONELY ELEPHANT,12


* agrupar por full name y agrupar por número de películas

In [6]:
q = """
WITH t1 AS (
      SELECT first_name || " " || last_name as full_name,
        count(film.film_id) as movie_count,
        dense_rank() OVER (ORDER BY COUNT(film.film_id) desc) as d_rank
        FROM film
        INNER JOIN film_actor 
        ON film.film_id = film_actor.film_id
        INNER JOIN actor
        ON actor.actor_id = film_actor.actor_id
        GROUP BY full_name
        ORDER BY 2 DESC
      )
  
  
  
  SELECT 
    full_name as nombre
    ,movie_count as conteo
    ,d_rank as rank 
  FROM t1
  WHERE d_rank <= 10;
    """
df = pd.read_sql(q, conn)
df

,nombre,conteo,rank
0,SUSAN DAVIS,54,1
1,GINA DEGENERES,42,2
2,WALTER TORN,41,3
3,MARY KEITEL,40,4
4,MATTHEW CARREY,39,5
5,SANDRA KILMER,37,6
6,SCARLETT DAMON,36,7
7,VIVIEN BASINGER,35,8
8,VAL BOLGER,35,8
9,UMA WOOD,35,8


In [7]:
df.columns

Index(['nombre', 'conteo', 'rank'], dtype='object')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   nombre  30 non-null     object
 1   conteo  30 non-null     int64 
 2   rank    30 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 848.0+ bytes


In [9]:
df_new=df.loc[df['rank']>5]
#copia en df_new el df original con rank>5

* veces que una pelicula ha pasado de disponible a alquiler y mostrar el precio

#### REVISAR

In [10]:
q=""" 
SELECT * FROM RENTAL
WHERE return_date is Null OR (JULIANDAY(return_date)-JULIANDAY(rental_date)>7)
"""
df = pd.read_sql(q, conn)
df

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,3,2005-05-24 23:03:39.000,1711,408,2005-06-01 22:12:39.000,1,2023-05-27 11:19:22
1,4,2005-05-24 23:04:41.000,2452,333,2005-06-03 01:43:41.000,2,2023-05-27 11:19:22
2,5,2005-05-24 23:05:21.000,2079,222,2005-06-02 04:33:21.000,1,2023-05-27 11:19:22
3,11,2005-05-25 00:09:02.000,4443,142,2005-06-02 20:56:02.000,2,2023-05-27 11:19:22
4,15,2005-05-25 00:39:22.000,3049,319,2005-06-03 03:30:22.000,1,2023-05-27 11:19:22
...,...,...,...,...,...,...,...
4672,16037,2005-08-23 22:13:04.000,341,45,2005-09-01 02:48:04.000,2,2023-05-27 11:20:38
4673,16039,2005-08-23 22:18:51.000,545,78,2005-08-31 19:55:51.000,2,2023-05-27 11:20:38
4674,16040,2005-08-23 22:19:33.000,3524,195,2005-09-02 02:19:33.000,2,2023-05-27 11:20:38
4675,16043,2005-08-23 22:21:03.000,3869,526,2005-08-31 03:09:03.000,2,2023-05-27 11:20:38


* clientes que no han devuelto un dvd

In [11]:
q=""" 
SELECT 
    c.first_name
    ,c.last_name
    ,c.email
    ,c.active
    ,COUNT (r.rental_id) as no_devueltos
FROM customer as c
INNER JOIN rental as r
ON c.customer_id=r.customer_id
WHERE r.return_date IS NULL and c.active=0
GROUP BY R.customer_id
HAVING no_devueltos >= 1
ORDER BY 3
"""
df = pd.read_sql(q, conn)
df

,first_name,last_name,email,active,no_devueltos
0,CHRISTIAN,JUNG,CHRISTIAN.JUNG@sakilacustomer.org,0,1
1,JUDITH,COX,JUDITH.COX@sakilacustomer.org,0,1
2,KENNETH,GOODEN,KENNETH.GOODEN@sakilacustomer.org,0,1
3,TERRANCE,ROUSH,TERRANCE.ROUSH@sakilacustomer.org,0,1


##### 1. ¿Tenemos actores en la tabla de actores que comparten el nombre completo y, en caso afirmativo, muestran esos nombres compartidos?

In [12]:
q=(""" 
 SELECT actor_id, first_name, last_name
 FROM actor
 WHERE (first_name, last_name) 
    in(
    SELECT first_name, last_name
    FROM actor
    GROUP BY first_name, last_name
    HAVING COUNT (actor_id)>1
    )
"""
)
df=pd.read_sql(q,conn)
df

,actor_id,first_name,last_name
0,101,SUSAN,DAVIS
1,110,SUSAN,DAVIS


In [13]:
## otra forma
q=""" 
SELECT
DISTINCT a1.first_name, a1.last_name
FROM actor as a1 JOIN actor as a2
WHERE (a1.first_name=a2.first_name) AND (a1.last_name=a2.last_name) AND (a1.actor_id!=a2.actor_id)
"""
df = pd.read_sql(q, conn)
df

,first_name,last_name
0,SUSAN,DAVIS


In [14]:
q=""" 
SELECT
    COUNT(DISTINCT first_name||" "|| last_name) as Nombres_distintos
FROM actor;
"""
df = pd.read_sql(q, conn)
df

,Nombres_distintos
0,199


##### 2. Muestra los nombres de los clientes que comparten la misma dirección (por ejemplo, esposo y esposa)
Para responder a esta pregunta, debemos buscar más de un cliente que tenga el mismo address_id pero diferentes customer_ids. Eso significa que haremos JOIN a la tabla de clientes para sí mismo (autounión).

>> REVISAR. No encuentro coincidencias

In [15]:
q="""
select 
count(address_id) from address
group by address_id
order by address_id
"""

df = pd.read_sql(q, conn)
df

,count(address_id)
0,1
1,1
2,1
3,1
4,1
...,...
598,1
599,1
600,1
601,1


In [16]:
q="""
select 
address_id, count(address_id) from customer
group by address_id
order by address_id
"""

df = pd.read_sql(q, conn)
df

,address_id,count(address_id)
0,5,1
1,6,1
2,7,1
3,8,1
4,9,1
...,...,...
594,601,1
595,602,1
596,603,1
597,604,1


##### 3. Muestra el monto total pagado por todos los clientes en la tabla de pagos.
Utilizar aquí la función agregada SUM() en la columna de monto de la tabla de pagos

In [17]:
q=""" 
SELECT SUM(amount) as Total_pagado /*Suma de columna amount*/
FROM payment
"""
df = pd.read_sql(q, conn)
df

,Total_pagado
0,67416.51


##### 4. Muestre el monto total pagado por cada cliente en la tabla de pagos.

In [18]:
q=""" 
SELECT c.first_name||" "||c.last_name as Nombre, SUM(p.amount) as Total_pagado
FROM payment as p
INNER JOIN customer as c
WHERE c.customer_id=p.customer_id
GROUP BY c.customer_id /*agrupar por cliente*/
"""
df = pd.read_sql(q, conn)
df

,Nombre,Total_pagado
0,MARY SMITH,118.68
1,PATRICIA JOHNSON,128.73
2,LINDA WILLIAMS,135.74
3,BARBARA JONES,81.78
4,ELIZABETH BROWN,144.62
...,...,...
594,TERRENCE GUNDERSON,117.70
595,ENRIQUE FORSYTHE,96.72
596,FREDDIE DUGGAN,99.75
597,WADE DELVALLE,83.78


**5. ¿Cuál es el pago total más alto realizado?**

In [19]:
q=""" 
SELECT SUM(p.amount) as Pago_mas_alto
FROM payment as p
INNER JOIN customer as c
WHERE c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY SUM(p.amount) DESC /*ordenar por Pago_mas_alto*/
LIMIT 1 /*mostrar 1*/
"""
df = pd.read_sql(q, conn)
df

,Pago_mas_alto
0,221.55


**6. ¿Cuál es el nombre del cliente que realizó los pagos totales más altos?**

In [20]:
q=""" 
SELECT c.first_name||" "||c.last_name as Nombre, SUM(p.amount) as Total_pagado
FROM payment as p
INNER JOIN customer as c
WHERE c.customer_id=p.customer_id
GROUP BY c.customer_id
ORDER BY SUM(p.amount) DESC
LIMIT 1
"""
df = pd.read_sql(q, conn)
df

,Nombre,Total_pagado
0,KARL SEAL,221.55


**7. ¿Cuáles son las películas que más se alquilaron?**

In [21]:
q=""" 
SELECT 
    title as Titulo
    ,f.film_id
    ,count(rental_id) as Alquileres
FROM
    rental as r
    ,inventory as i
    ,film as f
WHERE
    r.inventory_id=i.inventory_id AND f.film_id=i.film_id
GROUP BY title
ORDER BY Alquileres DESc
LIMIT 10
"""
df = pd.read_sql(q, conn)
df

,Titulo,film_id,Alquileres
0,BUCKET BROTHERHOOD,103,34
1,ROCKETEER MOTHER,738,33
2,SCALAWAG DUCK,767,32
3,RIDGEMONT SUBMARINE,730,32
4,JUGGLER HARDLY,489,32
5,GRIT CLOCKWORK,382,32
6,FORWARD TEMPLE,331,32
7,ZORRO ARK,1000,31
8,WIFE TURN,973,31
9,TIMBERLAND SKY,891,31


**8. Qué películas se han alquilado hasta ahora.**

Debemos entender que no es necesario que ya se hayan alquilado todas las películas de la tabla de películas.


In [22]:
q=""" 
SELECT 
    title as Titulo
    ,f.film_id
    ,count(rental_id) as Alquileres
FROM
    rental as r
    ,inventory as i
    ,film as f
WHERE
    r.inventory_id=i.inventory_id AND f.film_id=i.film_id
GROUP BY title
ORDER BY title
"""
df = pd.read_sql(q, conn)
peliculas_alquiladas=df
df

,Titulo,film_id,Alquileres
0,ACADEMY DINOSAUR,1,23
1,ACE GOLDFINGER,2,7
2,ADAPTATION HOLES,3,12
3,AFFAIR PREJUDICE,4,23
4,AFRICAN EGG,5,12
...,...,...,...
953,YOUNG LANGUAGE,996,7
954,YOUTH KICK,997,6
955,ZHIVAGO CORE,998,9
956,ZOOLANDER FICTION,999,17


**9. Qué películas no se han alquilado hasta ahora.**

In [16]:
#total películas - peliculas alquiladas

# peliculas_alquiladas

q=""" 
SELECT title, inventory_id
FROM film as f, inventory as i
WHERE f.film_id=i.film_id 
    AND i.inventory_id NOT IN(
        SELECT inventory_id
        FROM rental
        )
"""
df = pd.read_sql(q, conn)
df

#De las 5 de inventario de ACADEMY DINOSAUR, la numero 5 no se ha alquilado

,title,inventory_id
0,ACADEMY DINOSAUR,5


**10.  Qué clientes no han alquilado ninguna película hasta ahora.**

Un cliente podría haberse registrado como cliente del lugar de alquiler de DVD, pero aún no ha comenzado a alquilar DVD.

In [27]:
q=""" 
SELECT customer_id, count(customer_id) as "Alquileres por cliente"
from rental
group by customer_id
order by "Alquileres por cliente"
"""
df = pd.read_sql(q, conn)
df

,customer_id,Alquileres por cliente
0,318,12
1,61,14
2,110,14
3,281,14
4,136,15
...,...,...
594,75,41
595,144,42
596,236,42
597,526,45


In [17]:
# Clientes cuyo Id no está en rental

q=""" 
SELECT customer_id
FROM customer
WHERE customer_id NOT IN(
    SELECT customer_id
    FROM rental
)

"""
df = pd.read_sql(q, conn)
df

#todos los clientes han alquilado

,customer_id


**11. Muestra cada película y la cantidad de veces que se alquila.**

In [40]:
q=""" 
SELECT 
    title as Titulo
    ,count(rental_id) as Alquileres
FROM
    rental as r
    ,inventory as i
    ,film as f
WHERE
    r.inventory_id=i.inventory_id AND f.film_id=i.film_id
GROUP BY title
ORDER BY title
"""
df = pd.read_sql(q, conn)
df

,Titulo,Alquileres
0,ACADEMY DINOSAUR,23
1,ACE GOLDFINGER,7
2,ADAPTATION HOLES,12
3,AFFAIR PREJUDICE,23
4,AFRICAN EGG,12
...,...,...
953,YOUNG LANGUAGE,7
954,YOUTH KICK,6
955,ZHIVAGO CORE,9
956,ZOOLANDER FICTION,17


**12. Muestra la cantidad de películas en las que actuó cada actor.**

In [32]:
q=""" 
SELECT a.first_name||" "||a.last_name as Actor, COUNT(f.title) as Peliculas
FROM actor as a, film as f, film_actor as f2
WHERE a.actor_id=f2.actor_id AND f2.film_id=f.film_id
GROUP BY Actor
ORDER BY Actor
"""
df = pd.read_sql(q, conn)
df

,Actor,Peliculas
0,ADAM GRANT,18
1,ADAM HOPPER,22
2,AL GARLAND,26
3,ALAN DREYFUSS,27
4,ALBERT JOHANSSON,33
...,...,...
194,WILL WILSON,31
195,WILLIAM HACKMAN,27
196,WOODY HOFFMAN,31
197,WOODY JOLIE,31


**13. Muestre los nombres de los actores que actuaron en más de 20 películas.**

In [33]:
q=""" 
SELECT a.first_name||" "||a.last_name as Actor, COUNT(f.title) as Peliculas
FROM actor as a, film as f, film_actor as f2
WHERE a.actor_id=f2.actor_id AND f2.film_id=f.film_id
GROUP BY Actor
HAVING Peliculas>=20
ORDER BY title
"""
df = pd.read_sql(q, conn)
df

,Actor,Peliculas
0,CHRISTIAN GABLE,22
1,JOHNNY CAGE,29
2,LUCILLE TRACY,30
3,MARY KEITEL,40
4,MENA TEMPLE,30
...,...,...
184,GREG CHAPLIN,27
185,LIZA BERGMAN,25
186,MEG HAWKE,27
187,VIVIEN BERGEN,30


**14. ¿Cuántos actores tienen 8 letras solo en sus nombres?**

In [34]:
q=""" 
SELECT first_name||" "||last_name as "Nombre 8 letras"
FROM actor
WHERE LENGTH(first_name)=8
"""
df = pd.read_sql(q, conn)
df

,Nombre 8 letras
0,PENELOPE GUINESS
1,JENNIFER DAVIS
2,PENELOPE PINKETT
3,MICHELLE MCCONAUGHEY
4,ANGELINA ASTAIRE
5,SCARLETT DAMON
6,CHARLIZE DENCH
7,PENELOPE CRONYN
8,HARRISON BALE
9,PENELOPE MONROE


**15. Para todas las películas clasificadas como "PG", muéstrame la película y la cantidad de veces que se alquiló.**

Escribir una subconsulta para seleccionar todos los film_ids que se han alquilado hasta ahora. Para eso, se necesita unir tablas de alquiler e inventario en la columna común Inventory_id.

In [4]:
q=""" 
SELECT f.title, f.rating, f.film_id, count(r.rental_id)
FROM film as f, rental as r, inventory as i
WHERE f.rating="PG"
HAVING f.film_id=i.film_id AND i.inventory_id=r.inventory_id
ORDER BY f.title
"""
df = pd.read_sql(q, conn)
df

**16. Muestra las películas que se ofrecen para alquilar en store_id 1 y no se ofrecen en store_id 2.**

In [36]:
q=""" 
SELECT title, f.film_id, store_id
FROM film as f
LEFT JOIN inventory as i
WHERE f.film_id NOT IN(
    SELECT film_id 
    FROM inventory
    WHERE store_id=2
)
GROUP BY title
"""
df = pd.read_sql(q, conn)
df

,title,film_id,store_id
0,ALICE FANTASIA,14,1
1,AMELIE HELLFIGHTERS,20,1
2,ANALYZE HOOSIERS,24,1
3,ANONYMOUS HUMAN,27,1
4,ANTHEM LUKE,28,1
...,...,...,...
233,WON DARES,983,1
234,WONDERFUL DROP,984,1
235,WYOMING STORM,994,1
236,YOUNG LANGUAGE,996,1


**17. Muestre las películas que se ofrecen para alquilar en cualquiera de las dos tiendas 1 y 2.**


In [37]:
q=""" 
SELECT title, f.film_id
FROM film as f, inventory as i
WHERE i.store_id=1 OR i.store_id=2
GROUP BY title
"""
df = pd.read_sql(q, conn)
df

,title,film_id
0,ACADEMY DINOSAUR,1
1,ACE GOLDFINGER,2
2,ADAPTATION HOLES,3
3,AFFAIR PREJUDICE,4
4,AFRICAN EGG,5
...,...,...
995,YOUNG LANGUAGE,996
996,YOUTH KICK,997
997,ZHIVAGO CORE,998
998,ZOOLANDER FICTION,999


**18. Muestra los títulos de las películas que se ofrecen en ambas tiendas al mismo tiempo**

In [38]:
q=""" 
SELECT title, f.film_id, i.store_id
FROM film as f, inventory as i
WHERE i.store_id=1 AND i.store_id=2
"""
df = pd.read_sql(q, conn)
df

,title,film_id,store_id


**19. Para cada tienda, muestre el número de clientes que son miembros de esa tienda**

In [39]:
q=""" 
SELECT store_id as Tienda, count(store_id) as "Nº de clientes"
FROM customer
GROUP BY store_id
"""
df = pd.read_sql(q, conn)
df

,Tienda,Nº de clientes
0,1,326
1,2,273


**20. Muestra el título de la película más alquilada en la tienda con store_id 1**

In [12]:
q=""" 
SELECT title, COUNT(r.rental_id) as Alquileres
FROM film as f, inventory as i, rental as r
WHERE store_id=1 AND f.film_id=i.film_id AND r.inventory_id=i.inventory_id
GROUP BY title
ORDER BY Alquileres DESC
"""
df = pd.read_sql(q, conn)
df.head(1)

,title,Alquileres
0,LOVE SUICIDES,20


**21. Cuántas películas aún no se ofrecen para alquilar en las tiendas. Hay dos tiendas solo 1 y 2**

In [35]:
# film_id que no esten en inventario
q=""" 
SELECT film_id , title
FROM film
WHERE film_id NOT IN (
    SELECT film_id
    FROM inventory
)
"""
df = pd.read_sql(q, conn)
df

,film_id,title
0,14,ALICE FANTASIA
1,33,APOLLO TEEN
2,36,ARGONAUTS TOWN
3,38,ARK RIDGEMONT
4,41,ARSENIC INDEPENDENCE
5,87,BOONDOCK BALLROOM
6,108,BUTCH PANTHER
7,128,CATCH AMISTAD
8,144,CHINATOWN GLADIATOR
9,148,CHOCOLATE DUCK


**22. Muestre los customer_id para aquellos clientes que alquilaron un DVD de películas más de una vez.**

Para responder a esta pregunta, primero debemos conocer los film_ids que alquiló cada cliente.

In [28]:
#film_id alquilados por cada cliente

q=""" 
    SELECT customer_id, i.film_id
    FROM rental as r, inventory as i, film as f
    WHERE r.inventory_id=i.inventory_id AND i.film_id=f.film_id
    ORDER BY customer_id

"""
df = pd.read_sql(q, conn)
df

,customer_id,film_id
0,1,663
1,1,875
2,1,611
3,1,228
4,1,308
...,...,...
16039,599,1000
16040,599,891
16041,599,881
16042,599,869


In [47]:
q=""" 
SELECT r.customer_id, i.film_id, title, COUNT(r.inventory_id)
FROM film as f, inventory as i, rental as r
WHERE r.inventory_id=i.inventory_id AND f.film_id=i.film_id
group by i.inventory_id
ORDER BY title
"""
df = pd.read_sql(q, conn)
df

,customer_id,film_id,title,COUNT(r.inventory_id)
0,431,1,ACADEMY DINOSAUR,3
1,411,1,ACADEMY DINOSAUR,5
2,39,1,ACADEMY DINOSAUR,2
3,301,1,ACADEMY DINOSAUR,2
4,587,1,ACADEMY DINOSAUR,5
...,...,...,...,...
4575,12,1000,ZORRO ARK,5
4576,351,1000,ZORRO ARK,3
4577,198,1000,ZORRO ARK,5
4578,275,1000,ZORRO ARK,2


In [68]:
q=""" 
SELECT customer_id, inventory_id, COUNT(inventory_id) as Alquileres
FROM rental
GROUP BY inventory_id
ORDER BY customer_id, inventory_id, Alquileres
"""
df = pd.read_sql(q, conn)
df

,customer_id,inventory_id,Alquileres
0,1,197,4
1,1,1407,4
2,1,1443,3
3,1,2785,4
4,1,3021,5
...,...,...,...
4575,599,456,4
4576,599,914,5
4577,599,1615,3
4578,599,1788,3


**23. Muestra la cantidad de películas alquiladas en cada clasificación.**

Para responder a esta pregunta, primero debemos conocer los ID de película que se alquilaron (unir tablas de alquiler e inventario) y las calificaciones de cada película (unir tabla de películas).

In [32]:
q=""" 
SELECT rating, COUNT(rating)
FROM film as f, inventory as i, rental as r
WHERE r.inventory_id=i.inventory_id AND i.film_id=f.film_id
GROUP BY rating
"""
df = pd.read_sql(q, conn)
df

,rating,COUNT(rating)
0,G,2773
1,NC-17,3293
2,PG,3212
3,PG-13,3585
4,R,3181


**24. Muestre el beneficio de cada una de las tiendas 1 y 2.**

Para responder a esta pregunta, necesitamos saber la cantidad pagada y el Inventory_id para cada película alquilada al unirse (tablas de alquiler y pago) y el store_id para esa transacción de alquiler (unirse a la tabla de inventario).


In [39]:
q=""" 
SELECT store_id as Tienda, SUM(amount) as Ganancias
FROM payment as p, customer as c
WHERE p.customer_id=c.customer_id
GROUP BY store_id
"""
df = pd.read_sql(q, conn)
df

,Tienda,Ganancia
0,1,37001.52
1,2,30414.99


**25.  Muestra el beneficio de cada una de las tiendas 1 y 2 seguido del beneficio total de ambas tiendas**

In [47]:
q=""" 
SELECT store_id as Tienda, SUM(amount) as Ganancias
FROM payment as p, customer as c
WHERE p.customer_id=c.customer_id
GROUP BY store_id
"""
df1 = pd.read_sql(q, conn)
print(df1)

q=""" 
SELECT SUM(amount) as "Ganancia Total"
FROM payment
"""
df = pd.read_sql(q, conn)
print(df)

   Tienda  Ganancias
0       1   37001.52
1       2   30414.99
   Ganancia Total
0        67416.51


**26. Cuente la cantidad de actores cuyos nombres no comienzan con una "A"**

In [53]:
q=""" 
SELECT count(first_name)
FROM actor 
WHERE first_name NOT LIKE "A%"
"""
df = pd.read_sql(q, conn)
df

,count(first_name)
0,187


**27. Busque el nombre del actor que comience con "P" seguido de (una *e* o una *a*) y luego cualquier otra letra**

In [76]:
q=""" 
SELECT first_name||" "||last_name
FROM actor
WHERE 
    first_name LIKE "Pa%" OR 
    first_name LIKE "Pe%"
"""
df = pd.read_sql(q, conn)
df

,"first_name||"" ""||last_name"
0,PENELOPE GUINESS
1,PARKER GOLDBERG
2,PENELOPE PINKETT
3,PENELOPE CRONYN
4,PENELOPE MONROE


In [73]:
q=""" 
SELECT
    COUNT(CASE WHEN first_name LIKE 'Pa%' OR first_name LIKE 'Pe%' THEN 1 END) "Empieza por Pa o Pe"
FROM actor
"""
df = pd.read_sql(q, conn)
df


,Empieza por Pa o Pe
0,5


**28. Busque nombres de clientes que comiencen con "P" seguido de 5 letras**

In [60]:
q=""" 
SELECT DISTINCT first_name
FROM customer
WHERE 
    first_name LIKE "P_____"
"""
df = pd.read_sql(q, conn)
df

,first_name
0,PAMELA
1,PHILIP


**29. Busque actores con PaRkEr como su nombre e ignore las letras mayúsculas y minúsculas. Luego seleccione actores llamados PaRkEr y haga coincidir el caso de la letra**

In [63]:
q=""" 
SELECT first_name
FROM actor
WHERE LOWER(first_name) LIKE "PaRkEr"
"""
df = pd.read_sql(q, conn)
df

,first_name
0,PARKER


### **no es case sensitive**

In [62]:
q=""" 
SELECT first_name
FROM actor
WHERE first_name LIKE "PaRkEr"
"""
df = pd.read_sql(q, conn)
df

,first_name
0,PARKER


**30. Busque nombres de actores que comiencen con "P" seguidos de cualquier letra de la a a la e y luego de cualquier otra letra**

In [68]:
q=""" 
SELECT DISTINCT first_name
FROM actor
WHERE
    first_name LIKE "Pa%" OR
    first_name LIKE "Pb%" OR
    first_name LIKE "Pc%" OR
    first_name LIKE "Pd%" OR
    first_name LIKE "Pe%"
"""
df = pd.read_sql(q, conn)
df

,first_name
0,PENELOPE
1,PARKER
